# Notebook Details
Notebook to generate the Reddit Small Dataset. Similar to that of the python script.

In [5]:
%pip install convokit tqdm nlpaug transformers clean-text nltk tensorflow openai

Note: you may need to restart the kernel to use updated packages.


# Human-typed dataset

In [22]:
from convokit import Corpus, download
corpus = Corpus(filename=download("reddit-corpus-small"))

KeyboardInterrupt: 

In [ ]:
corpus.print_summary_stats()

In [13]:
def save(uttr_current, file_name):
  with open(file_name, "w+") as f:
    for utt in uttr_current:
        f.write(str(utt) + "\n------\n")

In [ ]:
utterances = []

for utt in corpus.iter_utterances():
    utterances.append(utt.text)
save(utterances, "reddit-corpus-small.txt")

## Pruning

In [ ]:
# use of clean-text to strip various un-needed information
# https://pypi.org/project/clean-text/

from cleantext import clean
import tqdm
import re

emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

for utt_idx in tqdm.tqdm(range(len(utterances))):
  utterances[utt_idx] = clean(utterances[utt_idx],
      fix_unicode=True,
      to_ascii=True,
      lower=True,
      no_urls=True,
      no_emails=True,
      no_phone_numbers=True,
      no_currency_symbols=True,
      replace_with_url="<URL>",
      replace_with_email="<EMAIL>",
      replace_with_phone_number="<PHONE>",
      replace_with_currency_symbol="<CUR>",
      lang="en"
  )
  utterances[utt_idx] = emoji_pattern.sub(r'', utterances[utt_idx]) # no emoji

In [ ]:
# Remove utterances < 20 characters and > 300 characters
print(len(utterances))
utterances = [utt for utt in utterances if len(utt) >= 3000]
print(len(utterances))

In [ ]:
# Remove repeated utterances
print(len(utterances))
utterances = list(set(utterances))
print(len(utterances))

In [ ]:
save(utterances, "post-pruned.txt")

Tokenization using nltk

In [ ]:
import nltk
from nltk.tokenize import word_tokenize

nltk.download('punkt')
tokenized_human = []
utterances_human = utterances

for utt_idx in tqdm.tqdm(range(len(utterances))):
  tokenized_human.append(' '.join(word_tokenize(utterances[utt_idx])))

In [ ]:
save(tokenized_human, "tokenized.txt")

# Load Files

In [7]:
utterances_human = []
with open("post-pruned.txt", "r") as f:
	lines = f.readlines()
	lines = '\n'.join(lines)
	utterances_human = lines.split('\n------\n')
print(len(utterances_human))

194315


# AI-generated dataset

Using GPT-2 from huggingface, we generate separate sub-datasets representing:
- Create
- Reply
In a 50% - 50% ratio

In [11]:
import tqdm
import openai
import random
import os

openai.api_key="API_KEY"
os.environ['AZURE_OPENAI_ENDPOINT']="API_ENDPOINT"
openai.api_type = "azure"
openai.api_version = "API_DATE"

utterances_ai_reply = []

# generate replies
msgs = []
for utterance_human_idx in tqdm.tqdm(range(2000)):
  # try-except as a cheeky way to continue generation if API call does not work
  while True:
    try:
      # 0-shot reply generation
      txt = f"""Generate a reply to this Reddit Post:
{utterances_human[int(random.random() * len(utterances_human))]}
"""
      # generate
      res = openai.chat.completions.create(
          model= "MODEL_NAME",
          messages=[
              {
                  "role": "user",
                  "content": txt
              }
          ],
          temperature=0.4,
          max_tokens=256,
          presence_penalty=-0.5
      )
      res = res.choices[0].message.content

      # prevent too many posts created that are actually just the AI aplogising
      if res == None or "I'm sorry, but I" in res:
        raise Exception("I'm sorry")

      utterances_ai_reply.append(res)
      break
    except Exception as e:
      print(f"Exception: {e}")


  0%|          | 8/2000 [00:09<40:07,  1.21s/it]

Exception: Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400}}


  1%|          | 14/2000 [00:23<1:06:44,  2.02s/it]

Exception: Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400}}


  4%|▎         | 71/2000 [01:45<42:08,  1.31s/it]  

...

 99%|█████████▉| 1988/2000 [56:11<00:17,  1.44s/it]

Exception: Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400}}


100%|█████████▉| 1999/2000 [56:32<00:02,  2.35s/it]

Exception: Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400}}


100%|██████████| 2000/2000 [56:35<00:00,  1.70s/it]


In [14]:
save(utterances_ai_reply, "ai_reply.txt")

In [ ]:
import tqdm
import random
import openai
from openai import OpenAI
from typing import cast

def add_utterance(txt, utterance):
  return f"""
  ### START POST ###
  {utterances_human[int(random.random() * len(utterances_human))]}
  ### END POST ###
  """ + txt

client = OpenAI(api_key="API_KEY")
utterances_ai_create = []

# generate creates
for utterance_human_idx in tqdm.tqdm(range(2000)):
  # 3-shot creation of a new Reddit Post based on other Reddit Posts
  txt = """
### START POST ###
"""

  for i in range(3):
    txt = add_utterance(txt, utterances_human[int(random.random() * len(utterances_human))])

  # try-except as a cheeky way to continue generation if API call does not work
  while True:
    try:
      # generate
      res = client.chat.completions.create(
          model= "gpt-35-turbo-16k",
          messages=[
              {
                  "role": "user",
                  "content": txt
              }
          ],
          temperature=0.6,
          max_tokens=256,
          presence_penalty=0.5
      )
      res = res.choices[0].message.content

      # prevent too many posts created that are actually just the AI aplogising
      if res == None or "I'm sorry, but I" in res:
        raise Exception()

      utterances_ai_create.append(res)
      break
    except:
      continue

In [ ]:
save(utterances_ai_create, "ai_create.txt")